# 0 - Imports and set up

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import difflib as dfl
import timeit
import os

import rasterio as rio
from rasterio.mask import mask
from pyproj import CRS
from shapely.geometry import Point
from shapely.geometry import shape

import fiona # needed?

pd.set_option('display.max_columns', None)


In [2]:
shape_path = '../../data/shapes/district/districts_17_19_clean.shp'
input_path = '../../data/input/'
output_path = '../../data/output/sprint1_analysis/'
shape_sc_path = '../../data/shapes/subcounty/uga_admbnda_adm3_UBOS_v5_cleaned [CB].shp'

# 1 - Get facility data

In [3]:
shape = gpd.read_file(shape_path)


In [4]:
for x in ['name_16', 'name_17', 'name_18','name_19']:
    print(len(shape[x].unique()))

116
122
128
135


## a - Extract the private for profit (PFP) facilities from MoH long list and locate them

In [5]:
hos_19 = pd.read_csv(input_path+'hospitals/original_data/MFL-Dec-19.csv',encoding = "cp1252")


In [6]:
# Note that the district names in this shaefile correspond to the column 'name_17' of the reference district shapefile

shape_sc = gpd.read_file(shape_sc_path)

In [7]:
# Isolate the facility data we are interested in 

hos_19_pfp = hos_19.loc[(hos_19['Ownership'] == 'PFP')&(hos_19['Operional Status'] == 'Functional')].copy()
hos_19_pfp.rename(columns = {'Sub county' : 'Sub_county'},inplace=True)
print ('We focus on PFP facilities, which account for a total of ' + str(len(hos_19_pfp)) + ' functionnal facilities')

We focus on PFP facilities, which account for a total of 3035 functionnal facilities


In [8]:
# Clean the district and facility names

hos_19_pfp['district_clean']=hos_19_pfp.copy().District.str.replace(' District','')
hos_19_pfp['sub_county_clean']=hos_19_pfp.copy().Sub_county.str.replace(' Subcounty','')
hos_19_pfp.drop(['District','Sub_county','Sub county UID','Health Facility UID','Operional Status'],axis=1,inplace=True)

In [9]:
# Create the matching codes

shape_sc['code']=shape_sc.ADM1_EN.apply(lambda x: x[0:5])+"/"+shape_sc['ADM3_EN']
hos_19_pfp['code']=hos_19_pfp.district_clean.apply(lambda x: x[0:5])+"/"+hos_19_pfp['sub_county_clean']
hos_19_pfp['code'] = hos_19_pfp['code'].str.upper() 


In [10]:
# Here I do th fuzzy matching 

# If time, improve performance and matching %


codes = []
certainty = []

for code in hos_19_pfp['code']:
    match_code = dfl.get_close_matches(code, shape_sc['code'],n=1,cutoff=0.69)
    if len(match_code) > 0:
        codes.append(match_code[0])
        score = dfl.SequenceMatcher(None, code, match_code[0]).ratio()
        certainty.append(score)
    else :
        codes.append(None)
        certainty.append(None)

hos_19_pfp['match_code'] = codes
hos_19_pfp['certainty'] = certainty

print('This gives us a percentage of null of ' + str(round((hos_19_pfp['match_code'].isnull().sum()/len(hos_19_pfp))*100,2)) + " %",
     '\nFor a total number of null values of ' + str(hos_19_pfp['match_code'].isnull().sum()))

This gives us a percentage of null of 8.83 % 
For a total number of null values of 268


In [11]:
# Add the corresponding PCODES from the shape_sc admin files


shape_sc_code_only = shape_sc[['code','ADM3_PCODE','geometry']].set_index('code')

hos_19_pfp=pd.merge(hos_19_pfp,shape_sc_code_only,how='left',left_on='match_code',right_on='code')



In [12]:
# Now clean the rows from null 'geometry' values - i.e. my failed matches - and exclude irrelevant facility types 

hos_19_pfp_clean = hos_19_pfp.loc[(hos_19_pfp['geometry'].notnull())
                                  &(hos_19_pfp['Level'] != 'No')
                                  &(hos_19_pfp['Level'] != 'Drug Shop')].copy()
len(hos_19_pfp_clean)

2701

In [13]:
# Extracting the lat and long coordinates

lon = []
lat = []

for hosp in list(hos_19_pfp_clean.index):
    x = hos_19_pfp_clean.loc[hosp,'geometry'].centroid.x
    y = hos_19_pfp_clean.loc[hosp,'geometry'].centroid.y
    lon.append(x)
    lat.append(y)

hos_19_pfp_clean['lon'] = lon
hos_19_pfp_clean['lat'] = lat

In [14]:
hos_pfp_merge = hos_19_pfp_clean[['Health Facility','Level','Ownership','lon','lat']].copy()

In [15]:
hos_pfp_merge.head()

,Health Facility,Level,Ownership,lon,lat
0,Arembwola HC II,HC II,PFP,33.616027,2.741836
1,Anzoa Medical Bureau HC III,HC III,PFP,31.787840,3.375797
4,Maaji C HC II,HC II,PFP,31.570216,3.100765
10,Alelluyah Joint Maternity Clinic,HC II,PFP,33.203181,2.299010
11,Ocan Community Clinic,HC II,PFP,33.197595,2.171717


## b - Extract the public and nfp hospital location from DHIS2 location data

In [16]:
hos_gvt_nfp_full = pd.read_csv(input_path+'hospitals/original_data/Facilities_DHIS2_20160412.csv')

hos_gvt_nfp = hos_gvt_nfp_full.loc[(hos_gvt_nfp_full.ownership != 'Private For Profit')&
                                   (hos_gvt_nfp_full.status != 'Non Functional'),
                                   ['name','type','ownership','Lat','Long']]

hos_gvt_nfp.columns = ['Health Facility', 'Level', 'Ownership','lat', 'lon']


## c - Build the hospital map

In [17]:
hosp=pd.concat([hos_gvt_nfp,hos_pfp_merge],ignore_index=True)

In [18]:
hosp['Level'].unique()

array(['HC III', 'General Hospital', 'HC II', 'Clinic', 'HC IV', nan,
       'NR Hospital', 'RR Hospital', 'Clinc', 'Hospital',
       'Special Clincs'], dtype=object)

In [19]:
# Around ~50 nan values for levels, which are at +95% HC II based on manual check, 

hosp['Level'].where(hosp['Level'].notnull(),'HC II',inplace=True)

In [20]:
Level_dict = {'HC II': 'Health Centre II', 
              'HC III': 'Health Centre III', 
              'Clinc': 'Clinic',
              'HC IV': 'Health Centre IV',
              'Special Clincs': 'Clinic',
              'General Hospital':'Hospital',
              'NR Hospital':'National Referral Hospital', 
              'RR Hospital':'Regional Referral Hospital'}

In [21]:
hosp.replace({'Level' : Level_dict },inplace=True)

In [22]:
hosp['Level'].unique()

array(['Health Centre III', 'Hospital', 'Health Centre II', 'Clinic',
       'Health Centre IV', 'National Referral Hospital',
       'Regional Referral Hospital'], dtype=object)

In [23]:
hosp.to_csv(output_path+'/hospital_map.csv')
hosp.to_csv(input_path+'hospitals//hospital_map.csv')

# 2 - Building the health coverage map

In [24]:
#pop_data = input_path+'/demographics/UG_2020_population.tif'
#shape_file = '../Data/Mapping layout/Admin3/uga_admbnda_adm3_UBOS_v5_cleaned [CB].shp'

## a - Get the population data

In [25]:
dem_path=input_path+'/demographics/UG_2020_population.tif'
raster = rio.open(dem_path)


In [26]:
dem_data = raster.read(1, masked = True)

## b - Adding this into sub-county boundaries

In [67]:
def mask_raster(raster_path, shape, indexes=1, crop=True, nodata=-9999):
    with rio.open(raster_path) as raster:
        out_data, out_transform = rio.mask.mask(raster, [shape], indexes=indexes, crop=crop, nodata=nodata)
        out_meta = raster.meta
    return (out_data, out_transform, out_meta)

In [68]:
total = []
for s in shape['geometry']:
    out_data, _, _ = mask_raster(dem_path, s)
    values = out_data.flatten()[out_data.flatten() > 0]
    # if empty array switch to np.nan to prevent warnings
    if values.size == 0:
        values = np.nan
    total.append(np.sum(values))

In [69]:
shape['total_pop'] = total

In [71]:
shape[['name_16','name_17','name_18','name_19','ADM1_PCODE','total_pop']]

,name_16,name_17,name_18,name_19,ADM1_PCODE,total_pop
0,MASAKA,MASAKA,MASAKA,MASAKA,UG105,339105.781250
1,PALLISA,BUTEBO,BUTEBO,BUTEBO,UG233,137689.218750
2,ALEBTONG,ALEBTONG,ALEBTONG,ALEBTONG,UG323,266153.593750
3,BUKEDEA,BUKEDEA,BUKEDEA,BUKEDEA,UG219,243917.718750
4,BUSIA,BUSIA,BUSIA,BUSIA,UG202,389940.906250
...,...,...,...,...,...,...
130,ARUA,ARUA,ARUA,MADI OKOLLO,UG303,163505.625000
131,KAABONG,KAABONG,KAABONG,KARENGA,UG318,45270.742188
132,MOYO,MOYO,MOYO,OBONGI,UG309,36650.414062
133,KABERAMAIDO,KABERAMAIDO,KABERAMAIDO,KALAKI,UG213,140819.875000


## c - Get the hospital data into a shapefile

In [31]:
# Getting this csv into a shapefile

# creating a geometry column 
geometry = [Point(xy) for xy in zip(hosp['lon'], hosp['lat'])]

# Coordinate reference system : WGS84
crs = CRS('epsg:4326')

# Creating a Geographic data frame 
h_df = gpd.GeoDataFrame(hosp, crs=crs, geometry=geometry)
h_df = h_df[['Health Facility','Level','Ownership' ,'geometry']]
h_df.rename({'Health Facility':'Health_Facility'},axis=1,inplace=True)

h_df.head()

,Health_Facility,Level,Ownership,geometry
0,Buikwe HC III,Health Centre III,Government,POINT (33.02732 0.34348)
1,Buikwe St. Charles Lwanga HOSPITAL,Hospital,Private Not For Profit,POINT (33.03181 0.33944)
2,Kasaku HC II,Health Centre II,NGO,POINT (32.89112 0.36330)
3,Busabaga HC III,Health Centre III,Government,POINT (32.90639 0.29694)
4,ENG BD Military Lugazi HC III,Health Centre III,Government,POINT (32.91223 0.37140)


## d - Calculate hospital coverage 

In [32]:
# Define the population served by different facility types 
# Based on page 7 of this : http://library.health.go.ug/sites/default/files/resources/National%20Health%20Facility%20Master%20List%202018_0.pdf

pop_served = {'Clinic' : 1000, 
              'Health Centre II' : 5000,
              'Health Centre III': 20000,
              'Health Centre IV': 100000,
              'Hospital' : 500000,
              'Regional Referral Hospital' : 2000000,
              'National Referral Hospital': 10000000}

# Tweak this to only account for larger facilities, from HC-IV onwards

pop_served_hc4 = {'Clinic' : 0, 
              'Health Centre II' : 0,
              'Health Centre III': 0,
              'Health Centre IV': 100000,
              'Hospital' : 500000,
              'Regional Referral Hospital' : 2000000,
              'National Referral Hospital': 10000000}

# List what admin level each facility size is referring to

significance = {'Clinic' : 'name_19', 
                  'Health Centre II' : 'name_19',
                  'Health Centre III': 'name_19',
                  'Health Centre IV': 'name_19',
                  'Hospital' : 'name_19',
                  'Regional Referral Hospital' : 'region',
                  'National Referral Hospital': 'country'}

In [33]:
# Get teh list of hopsital names (not unique) and add the coverage of each 

hospitals = list(h_df.Health_Facility.unique())
h_df['served'] = h_df['Level'].apply(lambda x: pop_served.get(x))

In [40]:
shape['country']= 'ug'

In [34]:
# Create new columns for the vars I am about to create 

shape['contribution'] = 0

shape['contribution_hc4'] = 0

for facility_type in pop_served.keys():
    shape[facility_type]=0
    
shape['contribution_pfp'] = 0

In [35]:
# Define the tools I'll combine to extract which point is in which polygon

# Gets one point and runs through which polygons include it
def get_complex_mask(point_list, df):
    masks = []
    for point in point_list:
        masks.append(df.geometry.contains(point))
    return combine_masks(masks)

# Return a single mask saying whether each polygons contains at least one of the points entrered as input of get_complex
def combine_masks(masks):
    final_mask = masks[0]
    for i in range(1, len(masks)):
        for m in range(len(masks[i])):
            final_mask[m] = final_mask[m] + masks[i][m]
    out = [True if x > 0 else False for x in final_mask]
    return out

In [41]:
for hospital in hospitals:# add [:1] to reduce that to one example
    df = h_df[h_df.Health_Facility == hospital].reset_index() # get only one hospital's data 
    
    h_type = df.loc[0, 'Level'] # type of the hospital to understand hospital serve count
    owner = int(df.loc[0, 'Ownership']=='PFP')
    sig = significance.get(h_type) # get the coverage area oh the hospital / gives us teh column name 
    
    mask = get_complex_mask(df.geometry.tolist(), shape) #get the mask for each hospital points 
    
    if sum(mask) < 1:# For hospitals with no location data 
        continue
    hosp_location = shape[mask] #Gives the full informationfor this hospital's location(s)
    area_name = hosp_location[sig].tolist() # Here retruns the name area for admin1/2/3 depedning on sig 
    location_mask = shape[sig].isin(area_name)
    m = location_mask
    
    shape.loc[m, 'contribution'] = shape.loc[m, 'contribution'] + (pop_served.get(h_type)//len(shape.loc[m]))
    shape.loc[m, 'contribution_hc4'] = shape.loc[m, 'contribution_hc4'] + (pop_served_hc4.get(h_type)//len(shape.loc[m]))
    shape.loc[m, 'contribution_pfp'] = shape.loc[m, 'contribution_pfp'] + ((pop_served.get(h_type)//len(shape.loc[m]))*owner)
    shape.loc[m,h_type]=shape.loc[m,h_type]+1


In [42]:
print('Regarding coverage: This',
      shape.contribution.sum(),
      'should be around the same as this',
      h_df[~pd.isna(h_df.geometry.x)].served.sum(),
      '\n While this should be lower than both',
      shape.contribution_hc4.sum(),
      '\n Regarding facility type counts: This',
      shape['Health Centre II'].sum(),
      'should be around the same as this',
      len(h_df[h_df['Level']=='Health Centre II']),
     '\n Regarding facility type counts: This',
      shape['Clinic'].sum(),
      'should be around the same as this',
      len(h_df[h_df['Level']=='Clinic']))

Regarding coverage: This 200038948 should be around the same as this 193910000 
 While this should be lower than both 150199948 
 Regarding facility type counts: This 4043 should be around the same as this 3958 
 Regarding facility type counts: This 541 should be around the same as this 500


## e - Calculate the ratios

In [43]:
shape['ratio']=shape['contribution']/shape['total_pop']
shape['ratio_hc4']=shape['contribution_hc4']/shape['total_pop']
shape['ratio_pfp']=shape['contribution_pfp']/shape['total_pop']
shape['ratio_gvt_nfp']=shape['ratio']-shape['ratio_pfp']

In [44]:
print(shape['ratio'].mean(),
      shape['ratio_hc4'].mean(),
      shape['ratio_pfp'].mean(),
      shape['ratio_gvt_nfp'].mean())

5.102267771283247 3.960395288915888 0.3574453890816278 4.744822382201619


In [45]:
shape['ratio_rank']=shape['ratio'].rank(ascending=True,pct=True)
shape['ratio_hc4_rank']=shape['ratio_hc4'].rank(ascending=True,pct=True)
shape['ratio_pfp_rank']=shape['ratio_pfp'].rank(ascending=True,pct=True)
shape['ratio_gvt_nfp_rank']=shape['ratio_gvt_nfp'].rank(ascending=True,pct=True)


In [46]:
shape_noshapes=shape.copy().drop(['geometry'],axis=1)
shape_noshapes.to_csv(output_path+'health_map.csv')

In [47]:
shape.describe()

,code_17,code_18,total_pop,contribution,contribution_hc4,Clinic,Health Centre II,Health Centre III,Health Centre IV,Hospital,Regional Referral Hospital,National Referral Hospital,contribution_pfp,ratio,ratio_hc4,ratio_pfp,ratio_gvt_nfp,ratio_rank,ratio_hc4_rank,ratio_pfp_rank,ratio_gvt_nfp_rank
count,135.000000,135.000000,1.350000e+02,1.350000e+02,1.350000e+02,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.0,1.350000e+02,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,275.340741,276.481481,3.122146e+05,1.481770e+06,1.112592e+06,4.007407,29.948148,10.777778,1.607407,1.192593,0.948148,2.0,2.801333e+05,5.102268,3.960395,0.357445,4.744822,0.503704,0.503704,0.503704,0.503704
std,108.264337,108.306975,3.022554e+05,2.163149e+06,1.459374e+06,12.336995,114.902510,13.974212,1.832934,2.447955,0.222554,0.0,1.455182e+06,2.934274,2.504281,0.951838,2.829014,0.289742,0.289742,0.288583,0.289742
min,101.000000,101.000000,3.665041e+04,4.631480e+05,2.481480e+05,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2.0,0.000000e+00,1.426787,0.964885,0.000000,1.273956,0.007407,0.007407,0.103704,0.007407
25%,207.500000,208.500000,1.748038e+05,7.091680e+05,4.703700e+05,0.000000,9.000000,6.000000,1.000000,0.000000,1.000000,2.0,1.000000e+03,3.043370,2.093394,0.006231,2.732923,0.255556,0.255556,0.255556,0.255556
50%,303.000000,304.000000,2.535178e+05,1.056966e+06,8.481480e+05,1.000000,15.000000,9.000000,1.000000,1.000000,1.000000,2.0,1.000000e+04,4.255939,3.182985,0.056762,3.969176,0.503704,0.503704,0.503704,0.503704
75%,401.500000,401.500000,3.474177e+05,1.429182e+06,1.098148e+06,2.500000,23.000000,12.000000,2.000000,1.000000,1.000000,2.0,6.550000e+04,6.237674,5.216477,0.223799,6.056438,0.751852,0.751852,0.751852,0.751852
max,431.000000,432.000000,2.977770e+06,2.086415e+07,1.354815e+07,122.000000,1295.000000,154.000000,17.000000,21.000000,1.000000,2.0,1.387100e+07,16.082402,12.020243,8.126391,15.990080,1.000000,1.000000,1.000000,1.000000


# 3 - Getting the poverty map

In [72]:
poverty = gpd.read_file(input_path+'/poverty/SubCountyShapefiles [from GIS WG]/Uganda2018SubcountyMultidimensionalPovertyConsensual.shp')
poverty.crs="epsg:4326"


In [73]:
print(set(poverty['DName2018'].unique()).difference(set(shape['name_18'].unique())),
      set(poverty['DName2019'].unique()).difference(set(shape['name_18'].unique())))

set() set()


In [74]:
total = []
for a in poverty['geometry']:
    out_data, _, _ = mask_raster(dem_path, a)
    values = out_data.flatten()[out_data.flatten() > 0]
    # if empty array switch to np.nan to prevent warnings
    if values.size == 0:
        values = np.nan
    total.append(np.sum(values))
poverty['total_pop']=total

In [75]:
poverty_map=poverty[['DName2019','DName2018','Subcounty','total_pop','HH_Poverty']].copy()
poverty_map.set_index(['DName2018','Subcounty'],inplace=True)

In [76]:
poverty_map['population_perc'] = poverty_map['total_pop']/ poverty_map.groupby('DName2018')['total_pop'].transform('sum')
poverty_map['poverty_rate']=poverty_map['population_perc']*poverty_map['HH_Poverty']
poverty_map.groupby('DName2018').sum()['poverty_rate']

DName2018
ABIM          0.555566
ADJUMANI      0.350280
AGAGO         0.528257
ALEBTONG      0.346668
AMOLATAR      0.370643
                ...   
SSEMBABULE    0.090085
TORORO        0.242042
WAKISO        0.017777
YUMBE         0.517921
ZOMBO         0.421657
Name: poverty_rate, Length: 128, dtype: float64

In [77]:
poverty_map.to_csv(output_path+'/poverty_map.csv')

# 4 - Get the demographics data 

# CONTINUE HERE

In [78]:
def get_data(file):
    total = []
    for x in shape['geometry']:
        out_data, _, _ = mask_raster(file, x)
        values = out_data.flatten()[out_data.flatten() > 0]
        if values.size == 0:
            values = np.nan
        total.append(np.sum(values))
    return total


def get_data(shape_df, file):
    total = []
    for s in shape['geometry']:
        out_data, _, _ = mask_raster(file, s)
        values = out_data.flatten()[out_data.flatten() > 0]
        if values.size == 0:
            values = np.nan
        total.append(np.sum(values))
    return total

In [79]:
#Loop through all the rasters files 
def compute_data(data_path, files_path):
    for file in files_path:
        url = data_path+'/%s'

        total =  get_data(url %file)
        shape[file]= total 
        

In [80]:
female_raster_path = input_path+'demographics/Female_Age'
female_files1 = os.listdir(female_raster_path)
male_raster_path = input_path+'demographics/Male_Age'
male_files1 = os.listdir(male_raster_path)

In [81]:
compute_data(female_raster_path,female_files1)
compute_data(male_raster_path,male_files1)

In [82]:
lst = ['uga_f_0_2020.tif','uga_f_1_2020.tif','uga_f_5_2020.tif',
       'uga_f_10_2020.tif', 'uga_f_15_2020.tif','uga_f_20_2020.tif', 
       'uga_f_25_2020.tif', 'uga_f_30_2020.tif','uga_f_35_2020.tif', 
       'uga_f_40_2020.tif', 'uga_f_45_2020.tif','uga_f_50_2020.tif',
       'uga_f_55_2020.tif', 'uga_f_60_2020.tif','uga_f_65_2020.tif', 
       'uga_f_70_2020.tif', 'uga_f_75_2020.tif','uga_f_80_2020.tif',
       
       'uga_m_0_2020.tif', 'uga_m_1_2020.tif','uga_m_5_2020.tif', 
       'uga_m_10_2020.tif','uga_m_15_2020.tif', 'uga_m_20_2020.tif',
       'uga_m_25_2020.tif','uga_m_30_2020.tif', 'uga_m_35_2020.tif', 
       'uga_m_40_2020.tif','uga_m_45_2020.tif','uga_m_50_2020.tif', 
       'uga_m_55_2020.tif','uga_m_60_2020.tif', 'uga_m_65_2020.tif', 
       'uga_m_70_2020.tif', 'uga_m_75_2020.tif', 'uga_m_80_2020.tif']

cols= ['uga_f_0','uga_f_1','uga_f_5', 'uga_f_10','uga_f_15','uga_f_20','uga_f_25','uga_f_30','uga_f_35',
       'uga_f_40', 'uga_f_45','uga_f_50','uga_f_55','uga_f_60','uga_f_65', 'uga_f_70','uga_f_75','uga_f_80',
       
       'uga_m_0', 'uga_m_1','uga_m_5','uga_m_10','uga_m_15','uga_m_20','uga_m_25', 'uga_m_30','uga_m_35', 
       'uga_m_40', 'uga_m_45','uga_m_50','uga_m_55','uga_m_60', 'uga_m_65','uga_m_70','uga_m_75','uga_m_80']


In [83]:
df = shape[lst].copy()
df.columns = cols # rename columns
df['district'] = shape['name_19']

In [84]:
#Unpivot the dataframe
df2= df.melt(id_vars=['ADM3_PCODE'], var_name='Gender_Age', value_name='pop_value')

KeyError: "The following 'id_vars' are not present in the DataFrame: ['ADM3_PCODE']"

In [ ]:
def gender(x):
    if x[4:5] == "f":
        return("female")
    elif x[4:5] == "m":
        return("male")

In [ ]:
df2['Age']=df2['Gender_Age'].apply(lambda x: x[6:])
df2['Gender']=df2['Gender_Age'].apply(lambda x: gender(x))

In [ ]:
df2.to_csv(output_path+'Total_Age_gender_stacked.csv', index=False)
df.to_csv(output_path+'/Total_Age_gender.csv', index=False)